# SQL Requisições de Pedidos

In [ ]:
with itens_processo as (
    select 
        processo,
        ano,
        tipo,
        item,
        dense_rank() over (
            partition by processo, ano 
            order by tipo, item
        ) as item_seq
    from cgc_vw_itensprocesso
),
itens_pedido as (
    select 
        cvi.pedido,
        cvi.ano,
        cvi.tipo,
        cvi.item,
        ip.item_seq
    from cgc_vw_itenspedido cvi
    join itens_processo ip 
      on cvi.processo = ip.processo
     and cvi.anoprocesso = ip.ano
     and cvi.tipo = ip.tipo
     and cvi.itemprocesso = ip.item
),
base as (
    select
        a.pedido id_requi,
        concat(to_char(a.pedido, 'fm000000/'), extract(year from a.ano)::int % 2000) as requi,
        b.codlocal codccusto,
        1 empresa,
        ip.item_seq item,
        sum(a.quantidade) as quan,
        sum(a.valor) as vaun,
        sum(a.total) as vato
    from pinhalpm.cgc_vw_itenspedidorecebido a
    join cgcpedidos b 
      on b.pedido = a.pedido and b.ano = a.ano
    join cgcrecebpedido c 
      on c.pedido = a.pedido and c.numero = a.numero
    left join itens_pedido ip 
      on ip.pedido = a.pedido 
     and ip.ano = a.ano 
     and ip.tipo = a.tipo 
     and ip.item = a.item
    where extract(year from a.ano) = 2025
    group by 
        a.pedido, a.ano, b.codlocal, ip.item_seq
)
select concat(
'insert into icadreq (id_requi, requi, codccusto, empresa, item, quan1, quan2, vaun1, vaun2, vato1, vato2, cadpro, destino) values (',
id_requi,',''',
requi,''',',
codccusto,',',
empresa,',',
item,',',
quan,',',
quan,',',
vaun,',',
vaun,',',
vato,',',
vato,',',
'(select cadpro from icadped where id_cadped = ', id_requi,'25 and item = ', item, '),',
'000000000'');'
)
from base b;